In [1]:
import os
import json
import tweepy

In [2]:
# Leitura das chaves

with open("keys.txt","r") as text:
    text = text.read()
    keys = json.loads(text)

print(keys.keys())

dict_keys(['API_KEY', 'API_KEY_SECRET', 'ACCESS_TOKEN', 'ACCESS_TOKEN_SECRET', 'BEARER_TOKEN'])


In [3]:
def creates_api():
    
    auth = tweepy.OAuth1UserHandler(
      keys['API_KEY'], 
      keys['API_KEY_SECRET'], 
      keys['ACCESS_TOKEN'], 
      keys['ACCESS_TOKEN_SECRET'])
    
    return tweepy.API(auth)

In [4]:
api = creates_api()

#### Verificação das credenciais

In [5]:
my_credentials = api.verify_credentials()
my_credentials

User(_api=<tweepy.api.API object at 0x7fcdd4111e20>, _json={'id': 1527271494379872256, 'id_str': '1527271494379872256', 'name': 'Gustavo', 'screen_name': 'pythonistabr', 'location': 'Dentro da matrix', 'description': 'Apenas um rapaz latino americano tentando extrair dados do twitter.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 4, 'listed_count': 0, 'created_at': 'Thu May 19 12:54:52 +0000 2022', 'favourites_count': 1, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 0, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1527275742060584960/lD5m452J_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.c

### testando a API com uma busca simples de tweets publicados com a hashtag pantanal

In [6]:
query = "#pantanal"
locale = ""
language = "pt"

In [7]:
tweets = api.search_tweets(q = query, lang = language, tweet_mode = "extended")

for tweet in tweets:
    try:
        print(tweet.retweeted_status.full_text)
        print("=====")
    except AttributeError:
        print(tweet.full_text)
        print("=====")

"Teu pai pode ter sido um homi bom, Juma, mas ele matou meu pai!!!!" (Muda)
"O QUE OCÊ DISSE?????" (Juma) #Pantanal https://t.co/ZFeda3nMYa
=====
o feminismo da guta não durou um dia sem uma refeição completa feita pela mãe pronta na mesa. impressionante #pantanal
=====
A reação da Juma 👀 #Pantanal https://t.co/0exlB51yLR
=====
A união sinistra entre Juma e Alcides é a única terceira via possível para o Brasil hoje
#Pantanal
=====
Ansiosos pra #ThorLoveAndThunder?

Tem PROMO C&amp;A 🥳

• Ganhe 20% de DESCONTO na 1ª compra CeA: use código de consultor PROMODEAMIGA + cupom NOVOMINHA

❗️Se NÃO for 1ª compra, ganhe 10% OFF usando SÓ código de consultor PROMODEAMIGA

#PowerCouple #pantanal #StaySelcaDay Mari lima https://t.co/BnNL0QFV3F
=====
o feminismo da guta não durou um dia sem uma refeição completa feita pela mãe pronta na mesa. impressionante #pantanal
=====
Alanis Guillen quebra o silêncio e revela se está namorando Jesuíta Barbosa. Confira! 

#famosos #pantanal #juma #jove 

https:

### capturando tweets com o uso de cursores

In [8]:
publicacoes = [] #lista para armazenar as bucas
query = '#pantanal' #parametro buscado

In [9]:
for status in tweepy.Cursor(api.search_tweets, q=query, lang="pt",
                            count=50).items(5): # busca 50 tweets em 5 páginas
    publicacoes.append(status)